In [ ]:
#!unzip '/content/drive/MyDrive/ML_Summer2023/leaf/data.zip' -d '/content/drive/MyDrive/ML_Summer2023/leaf/'

In [ ]:
import tensorflow as tf
import os
import random
from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.layers import Dropout, Dense, GlobalAveragePooling2D

In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator( rescale = 1.0/255.,
                                  rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [ ]:

# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/ML_Summer2023/leaf/data/train',
                                                 target_size = (224, 224),
                                                 batch_size = 64,
                                                 class_mode = 'categorical')

Found 8697 images belonging to 4 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/ML_Summer2023/leaf/data/test',
                                            target_size = (224, 224),
                                            batch_size = 64,
                                            class_mode = 'categorical')

Found 3732 images belonging to 4 classes.


In [ ]:
import os
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import Model


from tensorflow.keras.applications import MobileNetV2

#from tensorflow.keras.applications.MobileNetV2 import preprocess_input
from tensorflow.keras.preprocessing import image
from glob import glob


MobileNetV2 = MobileNetV2(input_shape=(224,224,3), weights='imagenet', include_top=False)

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
num_layers_to_freeze = 10
for layer in MobileNetV2.layers[:num_layers_to_freeze]:
    layer.trainable = False


In [ ]:
x=MobileNetV2.output
x= GlobalAveragePooling2D()(x)
x = Flatten()(x)
dropout_rate = 0.5
x = Dropout(dropout_rate)(x)
prediction = Dense(4, activation='softmax',kernel_regularizer='l2')(x)

# create a model object
model = Model(inputs=MobileNetV2.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(
  loss='categorical_crossentropy',
  optimizer=Adam(learning_rate=0.001),
  metrics=['accuracy']
)

In [ ]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/ML_Summer2023/leaf/best_model.h5',
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1
)

In [ ]:
r = model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs=50,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set),
    callbacks=[checkpoint]  # Add the checkpoint callback
)

<ipython-input-12-e1869cdef97c>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/50
136/136 [==============================] - ETA: 0s - loss: 0.3067 - accuracy: 0.9131 
Epoch 1: val_loss improved from inf to 10.10955, saving model to /content/drive/MyDrive/ML_Summer2023/leaf/best_model.h5
136/136 [==============================] - 3650s 26s/step - loss: 0.3067 - accuracy: 0.9131 - val_loss: 10.1096 - val_accuracy: 0.4957
Epoch 2/50
136/136 [==============================] - ETA: 0s - loss: 0.1543 - accuracy: 0.9655
Epoch 2: val_loss did not improve from 10.10955
136/136 [==============================] - 190s 1s/step - loss: 0.1543 - accuracy: 0.9655 - val_loss: 11.5390 - val_accuracy: 0.3923
Epoch 3/50
136/136 [==============================] - ETA: 0s - loss: 0.1126 - accuracy: 0.9715
Epoch 3: val_loss improved from 10.10955 to 9.07738, saving model to /content/drive/MyDrive/ML_Summer2023/leaf/best_model.h5
136/136 [==============================] - 192s 1s/step - loss: 0.1126 - accuracy: 0.9715 - val_loss: 9.0774 - val_accuracy: 0.4437
Epoch 4/50
136/13

In [ ]:
import tensorflow as tf


keras_model = tf.keras.models.load_model('/content/drive/MyDrive/ML_Summer2023/leaf/best_model.h5')


In [ ]:
# Convert the Keras model to a TFLite model
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
tflite_model = converter.convert()


In [ ]:
# Save the TFLite model to a file
with open('/content/drive/MyDrive/ML_Summer2023/leaf/converted_model.tflite', 'wb') as f:
    f.write(tflite_model)


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define the preprocess_image function based on your preprocessing requirements
def preprocess_image(image_path, target_size=(224, 224)):
    # Load the image and resize it
    img = load_img(image_path, target_size=target_size)
    # Convert the image to a NumPy array
    img_array = img_to_array(img)
    # Normalize pixel values to the range [0, 1]
    img_array = img_array / 255.0
    # Add an additional dimension to match the model's input shape
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Rest of your code (loading the TFLite model, performing inference, etc.)


In [ ]:
import tensorflow as tf
import numpy as np
import os

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path='/content/drive/MyDrive/ML_Summer2023/leaf/converted_model.tflite')
interpreter.allocate_tensors()

# Get input and output tensor details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Define class names based on subfolder names
class_names = sorted(os.listdir('/content/drive/MyDrive/ML_Summer2023/leaf/data/test'))  # List subfolder names

correct_predictions = 0
total_samples = 0

# Iterate through subfolders (classes)
for class_name in class_names:
    class_folder = os.path.join('/content/drive/MyDrive/ML_Summer2023/leaf/data/test', class_name)
    files = os.listdir(class_folder)

    for file_name in files:
        image_path = os.path.join(class_folder, file_name)
        image = preprocess_image(image_path)  # Preprocess the image

        # Perform inference
        interpreter.set_tensor(input_details[0]['index'], image)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])

        predicted_class = np.argmax(output)
        if predicted_class == class_names.index(class_name):
            correct_predictions += 1
        total_samples += 1

accuracy = correct_predictions / total_samples
print(f"Accuracy: {accuracy:.2f}")


In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import precision_score
import os

# Define the preprocess_image function based on your preprocessing requirements
def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalize to [0, 1]
    img_array = img_array.astype(np.float32)  # Convert to FLOAT32
    img_array = np.expand_dims(img_array, axis=0)
    return img_array


# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path='/content/drive/MyDrive/ML_Summer2023/leaf/converted_model.tflite')
interpreter.allocate_tensors()

# Get input and output tensor details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Define class names based on subfolder names
class_names = sorted(os.listdir('/content/drive/MyDrive/ML_Summer2023/leaf/data/test'))   # List subfolder names

true_labels = []
predicted_labels = []

# Iterate through subfolders (classes)
for class_name in class_names:
    class_folder = os.path.join('/content/drive/MyDrive/ML_Summer2023/leaf/data/test', class_name)
    files = os.listdir(class_folder)

    for file_name in files:
        image_path = os.path.join(class_folder, file_name)
        image = preprocess_image(image_path)  # Preprocess the image

        # Perform inference
        interpreter.set_tensor(input_details[0]['index'], image)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])

        predicted_class = np.argmax(output)
        true_labels.append(class_names.index(class_name))
        predicted_labels.append(predicted_class)

# Calculate precision for each class
precision = precision_score(true_labels, predicted_labels, average=None)

# Print precision for each class
for class_name, class_precision in zip(class_names, precision):
    print(f"Precision for {class_name}: {class_precision:.2f}")


Precision for Bacterial Spot: 0.98
Precision for Blight: 0.98
Precision for Healthy: 0.88
Precision for Leaf Spot: 0.97


# Without Quantization

In [ ]:
import os

# Directory containing test images
test_images_dir = '/content/drive/MyDrive/ML_Summer2023/leaf/data/test/Blight'

# Get list of image file paths in the directory
test_image_paths = [os.path.join(test_images_dir, filename) for filename in os.listdir(test_images_dir) if filename.endswith('.JPG')]

# Now you can use test_image_paths in the code


In [ ]:
import numpy as np
import tensorflow as tf
import time

# Load your TFLite model
interpreter = tf.lite.Interpreter(model_path='/content/drive/MyDrive/ML_Summer2023/leaf/converted_model.tflite')  # Load the .tflite model file
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load test dataset and preprocess images
# Assuming you have a list of image file paths in test_image_paths
test_images = [tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224)) for image_path in test_image_paths]
test_images = [tf.keras.preprocessing.image.img_to_array(image) for image in test_images]
test_images = [np.expand_dims(image, axis=0) for image in test_images]



# Perform warm-up inference
warmup_iterations = 10
for _ in range(warmup_iterations):
    interpreter.set_tensor(input_details[0]['index'], test_images[0])
    interpreter.invoke()


# Calculate inference time for all test images
inference_times = []
for test_image in test_images:
    start_time = time.time()
    interpreter.set_tensor(input_details[0]['index'], test_image)
    interpreter.invoke()
    _ = interpreter.get_tensor(output_details[0]['index'])
    end_time = time.time()
    inference_time = (end_time - start_time) *1000
    inference_times.append(inference_time)

average_inference_time = sum(inference_times) / len(inference_times)

print(f"Total inference time for {len(test_images)} test images: {sum(inference_times):.2f} milliseconds")
print(f"Average inference time per image: {average_inference_time:.2f} milliseconds")


Total inference time for 837 test images: 14133.64 milliseconds
Average inference time per image: 16.89 milliseconds


#Post_training Quantization

In [ ]:
import tensorflow as tf

# Load your trained .h5 model
model = tf.keras.models.load_model('/content/drive/MyDrive/ML_Summer2023/leaf/best_model.h5')

# Convert the model to a quantization-friendly format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the quantized model to a .tflite file
with open('/content/drive/MyDrive/ML_Summer2023/leaf/quantized_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Quantized model saved as quantized_model.tflite")


Quantized model saved as quantized_model.tflite


In [ ]:
import numpy as np
import tensorflow as tf
import time


interpreter = tf.lite.Interpreter(model_path='/content/drive/MyDrive/ML_Summer2023/leaf/quantized_model.tflite')
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

test_images = [tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224)) for image_path in test_image_paths]
test_images = [tf.keras.preprocessing.image.img_to_array(image) for image in test_images]
test_images = [np.expand_dims(image, axis=0) for image in test_images]

warmup_iterations = 10
for _ in range(warmup_iterations):
    interpreter.set_tensor(input_details[0]['index'], test_images[0])
    interpreter.invoke()

inference_times = []
for test_image in test_images:
    start_time = time.time()
    interpreter.set_tensor(input_details[0]['index'], test_image)
    interpreter.invoke()
    _ = interpreter.get_tensor(output_details[0]['index'])
    end_time = time.time()
    inference_time = (end_time - start_time) *1000
    inference_times.append(inference_time)

average_inference_time = sum(inference_times) / len(inference_times)

print(f"Total inference time for {len(test_images)} test images: {sum(inference_times):.2f} milliseconds")
print(f"Average inference time per image: {average_inference_time:.2f} milliseconds")


Total inference time for 837 test images: 31958.75 milliseconds
Average inference time per image: 38.18 milliseconds
